In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# 현재경로 확인
os.getcwd()

'C:\\실습자료\\part4'

In [3]:
# 데이터 불러오기
data = pd.read_csv("./data/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [4]:
'''
id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜 \nprice: 집 가격 (타겟변수)\nbedrooms: 주택 당 침실 개수\nbathrooms: 주택 당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County grading system 으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyr_renovated: 집이 리모델링 된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [5]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 21613 nVar: 14


## 의미가 없다고 판단되는 변수 제거

In [6]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

## 범주형 변수를 이진형 변수로 변환
- 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기 때문에 0, 1로 표현한다.
- 데이터에서 0, 1로 표현되어 있으므로 과정 생략

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [7]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Stats_Models)

In [8]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x, has_constant = 'add') # Bias 추가
sm_model = sm.OLS(train_y, sm_train_x) # 모델 구축
fitted_sm_model = sm_model.fit() # 학습 진행
fitted_sm_model.summary() # 학습 모델 구조 확인

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Sun, 08 Sep 2019   Prob (F-statistic):               0.00
Time:                        18:42:40   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
# 결과 확인
sm_test_x = sm.add_constant(test_x, has_constant = 'add') # 테스트 데이터에 Bias 추가
sm_model_predict = fitted_sm_model.predict(sm_test_x) # 테스트 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(sm_model_predict, test_y)))) # RMSE
print(fitted_sm_model.params) # 회귀계수

RMSE: 239804.29670858168
const           7.185671e+06
bathrooms       1.302689e+05
bedrooms       -2.224791e+03
condition       1.641020e+04
floors          1.946305e+03
grade           1.955909e+05
waterfront      7.555423e+05
yr_built       -4.300787e+03
yr_renovated    1.273246e+01
dtype: float64


## Bagging 한 결과가 일반적인 결과보다 좋은지 확인

In [10]:
import random
bagging_predict_result = [] # 빈 리스트 생성
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    print(len(set(random_data_index)))
    sm_train_x = train_x.iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 설명 변수
    sm_train_y = train_y.iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 종속 변수
    sm_train_x = sm.add_constant(sm_train_x, has_constant = 'add') # Bias 추가
    sm_model = sm.OLS(sm_train_y, sm_train_x) # 모델 구축
    fitted_sm_model = sm_model.fit() # 학습 진행
    
    sm_test_x = sm.add_constant(test_x, has_constant = 'add') # 테스트 데이터에 Bias 추가
    sm_model_predict = fitted_sm_model.predict(sm_test_x) # 테스트 데이터 예측
    bagging_predict_result.append(sm_model_predict) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장

9551
9564
9585
9582
9583
9537
9586
9554
9610
9566


In [11]:
bagging_predict_result[0] # 0 ~ 9, 10번의 예측을 하였기 때문에 10개의 결과가 생성

735      5.615749e+05
2830     7.174409e+05
4106     1.103425e+06
16218    1.465712e+06
19964    6.937055e+05
1227     3.882274e+05
18849    7.826119e+05
19369    4.940103e+05
20164    4.983517e+05
7139     5.400580e+05
2174     6.368405e+05
13342    4.145709e+05
15468    2.643231e+05
7662     2.816943e+05
16941    3.424839e+05
5304     1.258119e+06
6377     3.154256e+05
12319    1.031966e+06
18835    2.548376e+05
5337     6.040401e+05
3018     3.934356e+05
1807     1.300726e+06
10110    8.201192e+05
7609     5.782311e+05
20693    6.016787e+05
13958    5.690195e+05
8918     2.612581e+05
18059    5.168479e+04
16118    5.649451e+05
10366    6.481796e+05
             ...     
5511     3.183722e+05
19662    2.310643e+05
3079     3.852472e+05
9124     5.268983e+05
6389     1.119411e+06
13813    6.126734e+05
3016     7.347250e+05
12738    3.913473e+05
14372    2.570579e+05
12542    8.346100e+05
12376    8.113439e+05
4988     7.299310e+05
1653     5.497730e+05
20196    6.814663e+05
3422     3

In [24]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성
for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [25]:
bagging_predict

[560152.6536888707,
 705982.3358735429,
 1113259.6554618694,
 1469249.2892208404,
 697071.9008094582,
 387402.4420198503,
 782326.1853956779,
 481549.89607298176,
 498679.7277520133,
 538016.8641350867,
 640538.2496435677,
 408092.23040686344,
 269632.01818348735,
 280815.4294473951,
 338557.0383768444,
 1258021.8491218172,
 323540.46346569323,
 1031462.9142313383,
 262112.37360707502,
 601094.7761655084,
 392803.04457817564,
 1296966.0996161634,
 819965.6277091763,
 582391.8845012614,
 595879.2041132366,
 569701.503950817,
 264392.40132373,
 47359.76662959829,
 563410.6619498439,
 638502.7572759419,
 564448.147337138,
 458469.0542968938,
 551365.5839380196,
 682933.5771315661,
 408993.44510903675,
 869647.0586738412,
 936822.095647553,
 635637.3600926751,
 392779.328122837,
 1077027.759547772,
 455343.7873919038,
 150018.421381309,
 488010.935639199,
 221485.09780788998,
 66000.22826628508,
 -41898.043345182574,
 249476.8058362864,
 285048.49306810246,
 364688.7360712853,
 717328.9306

In [26]:
# 예측한 결과값들의 평균을 계산하여 실제 테스트 데이트의 타겟변수와 비교하여 성능 평가
print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y)))) # RMSE

RMSE: 239741.70728887763


## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Scikit-Learn)

In [27]:
from sklearn.linear_model import LinearRegression
regression_model = LinearRegression() # 선형 회귀 모형
linear_model1 = regression_model.fit(train_x, train_y) # 학습 데이터를 선형 회귀 모형에 적합
predict1 = linear_model1.predict(test_x) # 학습된 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 239804.29670858147


## Bagging 을 이용하여 선형 회귀 모형에 적합 후 평가 (Sampling 10번)

In [28]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regression_model, # 선형회귀모형
                                 n_estimators = 5, # 5번 샘플링
                                 verbose = 1) # 학습 과정 표시
linear_model2 = bagging_model.fit(train_x, train_y) # 학습 진행
predict2 = linear_model2.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 239739.58738488657


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 그렇다면 Sampling을 많이 해보자!

In [29]:
bagging_model2 = BaggingRegressor(base_estimator = regression_model, # 선형 회귀모형
                                  n_estimators = 30, # 30번 샘플링
                                  verbose = 1) # 학습 과정 표시
linear_model3 = bagging_model2.fit(train_x, train_y) # 학습 진행
predict3 = linear_model3.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

RMSE: 239859.76201673783


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## 학습 데이터를 의사결정나무모형에 적합 후 평가 데이터로 검증

In [30]:
from sklearn.tree import DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
tree_model1 = decision_tree_model.fit(train_x, train_y) # 학습 데이터를 의사결정나무 모형에 적합
predict1 = tree_model1.predict(test_x) # 학습된 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 295224.8590001005


In [31]:
import random
bagging_predict_result = [] # 빈 리스트 생성
for _ in range(30):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    print(len(set(random_data_index)))
    sm_train_x = train_x.iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 설명 변수
    sm_train_y = train_y.iloc[random_data_index, ] # 랜덤 인덱스에 해당되는 학습 데이터 중 종속 변수
    decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
    tree_model1 = decision_tree_model.fit(sm_train_x, sm_train_y) # 학습 데이터를 의사결정나무 모형에 적합
 
    predict1 = tree_model1.predict(test_x) # 테스트 데이터 예측
    bagging_predict_result.append(predict1) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장

9607
9493
9569
9565
9556
9533
9562
9550
9497
9603
9601
9532
9544
9545
9517
9605
9594
9589
9501
9562
9550
9550
9538
9616
9595
9598
9494
9571
9577
9630


In [52]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성
for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [53]:
# 예측한 결과값들의 평균을 계산하여 실제 테스트 데이트의 타겟변수와 비교하여 성능 평가

print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y)))) # RMSE

RMSE: 233259.93468284846


In [54]:
bagging_predict_result[29]

array([ 324900. ,  460000. ,  843500. , ...,  290675.5, 1120000. ,
        664500. ])

## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (Sampling 10번)

In [55]:
bagging_decision_tree_model1 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 5, # 5번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model2 = bagging_decision_tree_model1.fit(train_x, train_y) # 학습 진행
predict2 = tree_model2.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 252609.51325974855


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [56]:
bagging_decision_tree_model2 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 30, # 30번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model3 = bagging_decision_tree_model2.fit(train_x, train_y) # 학습 진행
predict3 = tree_model3.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

RMSE: 232400.83983065168


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
